In [6]:
import openai
from dotenv import load_dotenv
import os
import random
import re

# ===== 환경 설정 =====
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()

TOTAL_QUESTIONS = 3

SKIP_KEYWORDS = [
    "다음 질문", "다음질문", "스킵", "skip", "pass", "next",
    "next question", "넘어가기", "넘어가자", "넘어갈게요"
]

# 태도/모범/성실 칭찬만 제거
EXCLUDE_KEYWORDS = [
    "성실", "성실함", "성실하게", "모범", "모범적인",
    "태도", "태도가", "자세가 좋", "정성을 다해",
    "발표한 모습이 보기 좋았", "발표를 잘함", "발표를 잘 하",
    "참여도가 높", "열심히 참여", "적극적으로 참여",
    "친구들과 잘 지내", "봉사정신이 투철", "예의바른 태도"
]

# 희망 분야 매핑
CAREER_SUBJECT_MAP = {
    "공학": ["수학", "기하", "미적", "미적분", "과학", "물리",
           "융합과학", "정보", "프로그래밍", "공학", "인공지능 수학"],
    "자연": ["생명", "생명과학", "화학", "지구", "지구과학", "물리", "과학"],
    "의학": ["생명", "생명과학", "화학", "보건", "의학"],
    "컴퓨터": ["정보", "프로그래밍", "AI", "데이터", "수학",
            "기하", "미적분", "인공지능 수학", "컴퓨터공학"],
    "소프트웨어": ["정보", "프로그래밍", "AI", "컴퓨터", "수학"],
    "AI": ["정보", "프로그래밍", "AI", "융합과학", "수학", "인공지능 수학"],
    "상경": ["경제", "사회", "정치", "수학", "확률과통계"],
    "경영": ["경영", "경제", "사회", "수학"],
    "인문": ["국어", "문학", "독서", "사회", "윤리", "철학"],
    "교육": ["교육", "심리", "국어", "사회"]
}

##########################################################
# 1. 전체 텍스트 합치기
##########################################################
def get_full_text(student_data):
    records = student_data.get("academic_records", [])
    if isinstance(records, list):
        full = "\n".join(str(x) for x in records)
    else:
        full = str(records)

    reading = student_data.get("reading", "")
    if reading:
        full += "\n" + str(reading)

    return full


##########################################################
# 2. 희망분야 → 학년별 자동 배정
##########################################################
def extract_career_by_grade(full_text):
    matches = re.findall(r"희망\s*분야\s*([^\n]+)", full_text)

    grade_raw = {1: None, 2: None, 3: None}
    for idx, raw in enumerate(matches[:3], start=1):
        cleaned = raw.replace("분야", "").replace("계열", "").strip()
        grade_raw[idx] = cleaned

    def normalize(field):
        if not field:
            return ""
        if "컴퓨터" in field or "소프트웨어" in field:
            return "컴퓨터"
        if "ai" in field.lower():
            return "AI"
        if "공학" in field:
            return "공학"
        if "자연" in field:
            return "자연"
        if "의학" in field:
            return "의학"
        if "경영" in field:
            return "경영"
        if "상경" in field:
            return "상경"
        if "인문" in field:
            return "인문"
        if "교육" in field:
            return "교육"
        return field

    grade_norm = {g: normalize(v) for g, v in grade_raw.items()}
    return grade_raw, grade_norm


##########################################################
# 3. 세특/창체 출처 자동 추출
##########################################################
def extract_sources(full_text):
    sources = []

    # 3-1) 세부능력특기사항
    pattern = re.compile(
        r"([가-힣A-Za-z0-9\s]+):\s*(.+?)(?=\n[가-힣A-Za-z0-9\s]+:|\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
        re.DOTALL
    )

    for m in pattern.finditer(full_text):
        subject = m.group(1).strip()
        desc = m.group(2).strip().replace("\n", " ")[:250]
        label = f"{subject}(세부능력특기사항)"
        sources.append((label, desc))

    # 3-2) 창체
    blocks = ["동아리활동", "자율활동", "진로활동", "봉사활동"]
    for b in blocks:
        bpat = re.compile(
            b + r"\s*\n(.+?)(?=\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
            re.DOTALL
        )
        for m in bpat.finditer(full_text):
            desc = m.group(1).strip().replace("\n", " ")[:250]
            sources.append((b, desc))

    return sources


##########################################################
# 4. 태도성 내용 제거
##########################################################
def filter_out_attitude(sources):
    clean = []
    for label, text in sources:
        combo = label + " " + text
        if not any(bad in combo for bad in EXCLUDE_KEYWORDS):
            clean.append((label, text))
    return clean


##########################################################
# 5. label → 과목명/활동종류 분리
##########################################################
def split_label(label):
    """
    label 예:
    - '인공지능 수학(세부능력특기사항)'
    - '동아리활동'
    """
    if "(" in label:
        subject = label.split("(")[0]
        activity = label[label.find("(")+1:-1]
    else:
        subject = label
        activity = "창체활동"

    return subject.strip(), activity.strip()

##########################################################
# 6. 학습용 생기부 요약 (토큰 절약용)
##########################################################
def summarize_student_record(text, client):
    prompt = f"""
다음은 한 학생의 생활기록부이다.
면접관의 '기준 데이터'로 쓰일 수 있도록 핵심만 요약하라.

요약 규칙:
- 태도/성실/인성 칭찬 제거
- 전공 적합 활동 중심
- 탐구 주제, 프로젝트, 심화 내용 위주
- 최대 6줄 이내

[생활기록부]
{text}
"""

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 대학 입학사정관이다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400
    )

    return resp.choices[0].message.content.strip()


##########################################################
# 7. 여러 학생 생기부를 '학습용 컨텍스트'로 합치기 (면접 대상 제외)
##########################################################
def build_training_context(recordings_dir, target_filename, client):
    chunks = []

    for filename in os.listdir(recordings_dir):
        if not filename.endswith(".txt"):
            continue
        if filename == target_filename:
            continue

        filepath = os.path.join(recordings_dir, filename)
        student = load_student_txt(filepath)
        text = get_full_text(student)

        summary = summarize_student_record(text, client)

        chunks.append(f"[다른 학생 요약]\n{summary}")

    return "\n\n---\n\n".join(chunks)


##########################################################
# 8. 메인 로직
##########################################################
def start_ai_interview(target_student_data, training_context):
    # ✅ 면접 대상 학생 생기부만 사용
    full_text = get_full_text(target_student_data)

    # --- 희망분야 추출 ---
    career_raw, career_norm = extract_career_by_grade(full_text)

    print("\n=== 희망분야 인식 결과 ===")
    for g in [1, 2, 3]:
        print(f"{g}학년 → {career_raw.get(g)} (키워드: {career_norm.get(g)})")
    print("==========================\n")

    # --- 출처 추출 ---
    sources = extract_sources(full_text)
    sources = filter_out_attitude(sources)

    if not sources:
        print("출처 없음. TXT 구조 확인 필요.")
        return

    # --- A 모드 SYSTEM PROMPT ---
    system_prompt = f'''
너는 대한민국 최상위권 공대 면접관이다.

아래는 여러 학생들의 생활기록부 일부이다.
이 정보는 질문의 기준과 난이도를 학습하기 위한 참고 자료일 뿐이다.
이 학생들을 언급하거나 비교하지 말라.

[학습용 생기부 모음]
{training_context}

--------------------------------------------------

아래 학생이 실제 면접 대상이다.
모든 질문과 평가는 반드시 이 학생의 생활기록부만을 근거로 하라.

[면접 대상 학생 생기부]
{get_full_text(target_student_data)}

규칙:
- 수준이 낮으면 냉정히 지적
- 생기부에 없는 내용에 대한 답변은 혹평하라
- 점수 < 70점이면 반드시 [다시 답변 요청]을 붙여라
'''

    messages = [{"role": "system", "content": system_prompt}]

    question_num = 1

    while question_num <= TOTAL_QUESTIONS:

        # === 1) 학년 랜덤 선택 ===
        selected_grade = random.choice([1,2,3])
        selected_career_raw = career_raw[selected_grade]
        selected_career_norm = career_norm[selected_grade]

        # === 2) 해당 학년 희망분야 기반 필터 ===
        allowed_keywords = CAREER_SUBJECT_MAP.get(selected_career_norm, [])
        grade_sources = [
            s for s in sources if any(k in (s[0] + s[1]) for k in allowed_keywords)
        ]
        if not grade_sources:
            grade_sources = sources

        # === 3) 출처 선택 ===
        label, text = random.choice(grade_sources)
        subject_name, activity_type = split_label(label)

        is_last = (question_num == TOTAL_QUESTIONS)

        # === 4) 질문 생성 ===
        user_prompt = f'''
다음 정보를 기반으로 {"[마지막 질문]" if is_last else "[질문]"}을 생성하라.

출처 학년: {selected_grade}학년
과목명: {subject_name}
활동종류: {activity_type}
출처 전문: {label}
핵심 내용: {text}

해당 학년 희망분야: {selected_career_raw} (키워드: {selected_career_norm})

형식:
{"[마지막 질문]" if is_last else "[질문]"}
출처: {selected_grade}학년 · {subject_name} ({activity_type})
희망분야({selected_grade}학년): {selected_career_raw}
핵심 내용: {text}
질문:
'''
        messages.append({"role": "user", "content": user_prompt})

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=600
        )
        qtext = resp.choices[0].message.content
        print("\n" + qtext)
        messages.append({"role": "assistant", "content": qtext})

        # === 5) 답변 받기 ===
        answer = input("\n[학생 답변 또는 '다음 질문'] > ")

        if any(k in answer.lower() for k in SKIP_KEYWORDS):
            print("\n[안내] 다음 질문으로 넘어갑니다.\n")
            question_num += 1
            continue

        if answer.lower() in ("exit", "quit"):
            print("면접을 종료합니다.")
            break

        # === 6) 평가 ===
        eval_prompt = f'''
[학생 답변]
{answer}

출처 학년: {selected_grade}학년
과목명: {subject_name}
활동종류: {activity_type}
출처 전문: {label}
핵심 내용: {text}
희망분야({selected_grade}학년): {selected_career_raw}

A 모드로 매우 날카롭게 평가하라.
'''
        messages.append({"role": "user", "content": eval_prompt})

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=900
        )
        eval_text = resp.choices[0].message.content
        print("\n" + eval_text)
        messages.append({"role": "assistant", "content": eval_text})

        if "[다시 답변 요청]" in eval_text:
            continue

        question_num += 1

RECORDINGS_DIR = "recordings_lite"
TARGET_FILE = "202315060_3학년_휴먼AI공학전공_서현승_정시_censored.txt"

def load_student_txt(filepath):
    """
    생활기록부 TXT 파일을 기존 코드가 쓰는 student_data 구조로 변환
    """
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read().strip()

    # 이름 추출 (없으면 기본값)
    name_match = re.search(r"이름\s*[:：]\s*(.+)", text)
    name = name_match.group(1).strip() if name_match else "이름 미상"

    return {
        "student_info": {
            "name": name
        },
        # 기존 로직은 academic_records를 텍스트로 처리 가능
        "academic_records": text,
        "reading": ""
    }


##########################################################
# 실행
##########################################################
if __name__ == "__main__":

    TARGET_FILE = "target_student.txt"

    training_context = build_training_context(
        RECORDINGS_DIR,
        TARGET_FILE,
        client
    )

    target_path = os.path.join(RECORDINGS_DIR, TARGET_FILE)
    target_student_data = load_student_txt(target_path)

    print("\n🎯 면접 대상:", target_student_data["student_info"]["name"])
    print("📚 학습용 생기부 요약 완료\n")

    start_ai_interview(
        target_student_data,
        training_context
    )

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-ieq3S40n7w9a3M3JXtI33y8c on tokens per min (TPM): Limit 100000, Used 94478, Requested 20778. Please try again in 109h50m35.52s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}